# Data Transformation

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import os
import pandas as pd

base_path = os.path.dirname(os.getcwd())
base_path=base_path.replace("/code","")
file_path = os.path.join(base_path,os.getenv("RAWDATA"))
quarter_path = os.path.join(base_path,os.getenv("Q_DATA"))

sheet_name = "月" 

df = pd.read_excel(file_path, sheet_name=sheet_name)
data_set={}
table_names=df.columns
for num in range(1,len(table_names)):
    avg_lsit=[]
    for data_num in range(0,len(df[table_names[num]]),3):
        data_avg=df[table_names[num]][data_num]+df[table_names[num]][data_num+1]+df[table_names[num]][data_num+2]/3
        avg_lsit.append(data_avg)
    data_set[table_names[num]]=avg_lsit

df_gdp = pd.DataFrame.from_dict(data_set)
df_gdp.to_csv(quarter_path, index=False, float_format='%.2f')  

# Data Consolidation

In [13]:
final_path = os.path.join(base_path,os.getenv("TRAINDATA"))
sheet_name = "季" 

df = pd.read_excel(file_path, sheet_name=sheet_name)
df_t=pd.read_csv(quarter_path)

if len(df) == len(df_t):
    df_merged = pd.concat([df, df_t], axis=1)  
    print(df_merged.head())  
else:
    print("兩個 DataFrame 的行數不同，無法合併")

df_merged.to_csv(final_path, index=False, float_format='%.2f')  

       年份     GDP  平均匯率(元/美元)  國內投資毛額    投資率  工業生產指數  製造業銷售量指數  \
0  71年第1季  461774       37.91  102048  22.10   27.57     30.02   
1  71年第2季  488234       38.78  143589  29.41   30.35     33.54   
2  71年第3季  489504       39.65  117934  24.09   30.06     32.76   
3  71年第4季  498511       40.15  144192  28.92   29.86     32.70   
4  72年第1季  495361       39.97   99632  20.11   29.28     31.80   

   電力(企業)總用電量(十億度)  進口物價基本分類指數  出口物價基本分類指數     進口貿易總值     出口貿易總值  \
0             4.85      231.63      311.21  147089.00  135223.67   
1             5.46      231.87      315.17  172059.67  156827.00   
2             5.81      234.50      317.22  181502.67  148698.00   
3             5.66      234.53      315.38  168145.00  135928.00   
4             5.21      227.62      309.54  153110.00  131671.00   

   機械及電機設備進口值(十億元)  消費者物價指數   失業率  就業人數（千人）  總人口數(千人)  景氣領先指標綜合指數(點)  \
0              0.0   129.59  3.42  15826.00  42389.67          28.72   
1              0.0   130.53  4.12  15577.33  42570

# X & Y dataset

In [14]:
x_path = os.path.join(base_path,os.getenv("X_LABEL"))
y_path = os.path.join(base_path,os.getenv("Y_LABEL"))

In [15]:
# 將「年份」欄位拆解成「年份」與「季別」
df_merged[['年份', '季別']] = df_merged['年份'].str.extract(r'(?P<年份>\d+)年第(?P<季別>\d)季')

# 將年份轉成西元（例如民國71年 → 西元1982年）
df_merged['年份'] = df_merged['年份'].astype(int) + 1911
df_merged['季別'] = df_merged['季別'].astype(int)


# 調整欄位順序：把 年份 和 季別 放在前面
df_cols = ['年份', '季別'] + [col for col in df_merged.columns if col not in ['年份', '季別', 'GDP']]
df_merged = df_merged[['年份', '季別', 'GDP'] + df_cols[2:]]

In [16]:
# 儲存 y.csv（只有 年份、季別、GDP）
df_merged[['年份', '季別', 'GDP']].to_csv(y_path, index=False)

# 儲存 x.csv（年份、季別 + 其他欄位，不含 GDP）
df_merged.drop(columns=['GDP']).to_csv(x_path, index=False)

# Mix dataset

In [17]:
import pandas as pd

# 讀取 x 和 y 的資料
x_mix_gdp = pd.read_csv(x_path)
y_mix_gdp = pd.read_csv(y_path)

# 取出 GDP 欄位的 series
gdp_series = y_mix_gdp['GDP']

# 建立當季 gdp 欄位
x_mix_gdp['gdp'] = pd.NA

# 建立前三季 gdp 欄位
x_mix_gdp['gdp_t1'] = pd.NA
x_mix_gdp['gdp_t2'] = pd.NA
x_mix_gdp['gdp_t3'] = pd.NA



# 用迴圈填入前三季 GDP
for i in range(len(x_mix_gdp)):
    x_mix_gdp.at[i, 'gdp'] = gdp_series[i]
    if i >= 1:
        x_mix_gdp.at[i, 'gdp_t1'] = gdp_series[i - 1]
    if i >= 2:
        x_mix_gdp.at[i, 'gdp_t2'] = gdp_series[i - 2]
    if i >= 3:
        x_mix_gdp.at[i, 'gdp_t3'] = gdp_series[i - 3]

# 儲存結果
x_mix_path = os.path.join(base_path,os.getenv("X_MIX_LABEL"))
x_mix_gdp.to_csv(x_mix_path, index=False)


In [18]:
# x_mix
x_mix_gdp

,年份,季別,平均匯率(元/美元),國內投資毛額,投資率,工業生產指數,製造業銷售量指數,電力(企業)總用電量(十億度),進口物價基本分類指數,出口物價基本分類指數,...,消費者物價指數,失業率,就業人數（千人）,總人口數(千人),景氣領先指標綜合指數(點),景氣同時指標綜合指數(點),gdp,gdp_t1,gdp_t2,gdp_t3
0,1982,1,37.91,102048,22.10,27.57,30.02,4.85,231.63,311.21,...,129.59,3.42,15826.00,42389.67,28.72,30.21,461774,<NA>,<NA>,<NA>
1,1982,2,38.78,143589,29.41,30.35,33.54,5.46,231.87,315.17,...,130.53,4.12,15577.33,42570.33,28.81,29.00,488234,461774,<NA>,<NA>
2,1982,3,39.65,117934,24.09,30.06,32.76,5.81,234.50,317.22,...,133.10,5.75,16000.33,42742.00,28.50,28.87,489504,488234,461774,<NA>
3,1982,4,40.15,144192,28.92,29.86,32.70,5.66,234.53,315.38,...,132.11,6.41,16146.33,42952.67,29.05,29.58,498511,489504,488234,461774
4,1983,1,39.97,99632,20.11,29.28,31.80,5.21,227.62,309.54,...,132.95,7.15,16260.67,43145.67,31.46,30.78,495361,498511,489504,488234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2023,4,31.84,1471478,23.06,215.34,209.11,32.09,263.75,261.96,...,249.07,7.88,26996.00,54258.33,215.93,210.73,6380686,6016101,5710934,5489013
168,2024,1,31.45,1536850,25.15,201.14,198.12,29.21,257.70,256.74,...,249.37,7.83,27019.33,54276.00,224.50,216.45,6111775,6380686,6016101,5710934
169,2024,2,32.35,1748574,28.05,217.22,215.02,32.97,265.20,264.72,...,250.43,7.83,27021.67,54265.67,229.75,223.13,6233160,6111775,6380686,6016101
170,2024,3,32.30,1666148,25.88,232.00,224.60,35.49,264.42,264.66,...,252.41,8.07,27092.00,54252.33,233.64,228.73,6437479,6233160,6111775,6380686


In [19]:
# y_mix
y_mix_gdp

,年份,季別,GDP
0,1982,1,461774
1,1982,2,488234
2,1982,3,489504
3,1982,4,498511
4,1983,1,495361
...,...,...,...
167,2023,4,6380686
168,2024,1,6111775
169,2024,2,6233160
170,2024,3,6437479


In [20]:
df_merged

,年份,季別,GDP,平均匯率(元/美元),國內投資毛額,投資率,工業生產指數,製造業銷售量指數,電力(企業)總用電量(十億度),進口物價基本分類指數,出口物價基本分類指數,進口貿易總值,出口貿易總值,機械及電機設備進口值(十億元),消費者物價指數,失業率,就業人數（千人）,總人口數(千人),景氣領先指標綜合指數(點),景氣同時指標綜合指數(點)
0,1982,1,461774,37.91,102048,22.10,27.57,30.02,4.85,231.63,311.21,147089.00,135223.67,0.00,129.59,3.42,15826.00,42389.67,28.72,30.21
1,1982,2,488234,38.78,143589,29.41,30.35,33.54,5.46,231.87,315.17,172059.67,156827.00,0.00,130.53,4.12,15577.33,42570.33,28.81,29.00
2,1982,3,489504,39.65,117934,24.09,30.06,32.76,5.81,234.50,317.22,181502.67,148698.00,0.00,133.10,5.75,16000.33,42742.00,28.50,28.87
3,1982,4,498511,40.15,144192,28.92,29.86,32.70,5.66,234.53,315.38,168145.00,135928.00,0.00,132.11,6.41,16146.33,42952.67,29.05,29.58
4,1983,1,495361,39.97,99632,20.11,29.28,31.80,5.21,227.62,309.54,153110.00,131671.00,0.00,132.95,7.15,16260.67,43145.67,31.46,30.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2023,4,6380686,31.84,1471478,23.06,215.34,209.11,32.09,263.75,261.96,2853790.00,2235742.33,1049.92,249.07,7.88,26996.00,54258.33,215.93,210.73
168,2024,1,6111775,31.45,1536850,25.15,201.14,198.12,29.21,257.70,256.74,2579464.00,2162123.67,929.02,249.37,7.83,27019.33,54276.00,224.50,216.45
169,2024,2,6233160,32.35,1748574,28.05,217.22,215.02,32.97,265.20,264.72,2841894.00,2394735.00,1129.48,250.43,7.83,27021.67,54265.67,229.75,223.13
170,2024,3,6437479,32.30,1666148,25.88,232.00,224.60,35.49,264.42,264.66,3152048.67,2543414.00,1243.91,252.41,8.07,27092.00,54252.33,233.64,228.73
